# Working With Spark on Docker

## MAP REDUCE CALCULATIONS WITH DATAFRAME API AND SPARK SQL

In [41]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "2g").appName('spark_nfl_data').getOrCreate()

If you look at the NFL dataset, those missing values we have tried to skip in are "NA". By default, spark will treat empty or null as missing values, so here we need to let spark treat "NA" as the missing values.

In [42]:
df_nfl = spark.read.options(header=True, nullValue='NA', inferSchema=True).csv('./NFL_Play_by_Play_2009-2018_2.csv')
df_nfl.show(5)

+-------+----------+---------+---------+-------+------------+-------+-------------+-------------------+-------------------------+----------------------+----------------------+---------+----+----------+---------+------------+----------------+----------------+-------------+-------------+------------------+---------------------+------------------+---------+--------------+--------------+----------------+
|play_id|   game_id|home_team|away_team|posteam|posteam_type|defteam|side_of_field|          game_date|quarter_seconds_remaining|half_seconds_remaining|game_seconds_remaining|game_half|down|goal_to_go|play_type|yards_gained|total_home_score|total_away_score|posteam_score|defteam_score|score_differential|fourth_down_converted|fourth_down_failed|touchdown|pass_touchdown|rush_touchdown|return_touchdown|
+-------+----------+---------+---------+-------+------------+-------+-------------+-------------------+-------------------------+----------------------+----------------------+---------+----+--

In [43]:
#let's check and clean the data with dataframe
print(df_nfl.count())
print(df_nfl.distinct().count())

449371
446982


In [44]:
#drop duplicate rows
df_nfl = df_nfl.dropDuplicates()

In [45]:
#let's look at the data
df_nfl.printSchema()

root
 |-- play_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- posteam: string (nullable = true)
 |-- posteam_type: string (nullable = true)
 |-- defteam: string (nullable = true)
 |-- side_of_field: string (nullable = true)
 |-- game_date: timestamp (nullable = true)
 |-- quarter_seconds_remaining: integer (nullable = true)
 |-- half_seconds_remaining: double (nullable = true)
 |-- game_seconds_remaining: double (nullable = true)
 |-- game_half: string (nullable = true)
 |-- down: integer (nullable = true)
 |-- goal_to_go: integer (nullable = true)
 |-- play_type: string (nullable = true)
 |-- yards_gained: integer (nullable = true)
 |-- total_home_score: integer (nullable = true)
 |-- total_away_score: integer (nullable = true)
 |-- posteam_score: integer (nullable = true)
 |-- defteam_score: integer (nullable = true)
 |-- score_differential: integer (nullable = true)
 |-- fou

In [46]:
#take a look at the dta via pandas
import pandas as pd
#disable the row/column limits to not truncate the displayed data
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_nfl.limit(10).toPandas().head(10)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,down,goal_to_go,play_type,yards_gained,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,fourth_down_converted,fourth_down_failed,touchdown,pass_touchdown,rush_touchdown,return_touchdown
0,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,815,1715.0,3515.0,Half1,3.0,0,pass,0,0,0,0,0,0,0,0,0,0,0,0
1,584,2009091000,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,327,1227.0,3027.0,Half1,3.0,0,pass,-9,0,0,0,0,0,0,0,0,0,0,0
2,1215,2009091000,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,539,539.0,2339.0,Half1,1.0,0,run,1,0,0,0,0,0,0,0,0,0,0,0
3,1653,2009091000,PIT,TEN,PIT,home,TEN,PIT,2009-09-10,134,134.0,1934.0,Half1,1.0,0,pass,0,0,0,0,0,0,0,0,0,0,0,0
4,3414,2009091000,PIT,TEN,PIT,home,TEN,TEN,2009-09-10,356,356.0,356.0,Half2,3.0,0,pass,15,7,10,7,10,-3,0,0,0,0,0,0
5,1444,2009091305,HOU,NYJ,NYJ,away,HOU,HOU,2009-09-13,335,335.0,2135.0,Half1,1.0,0,run,-1,0,3,3,0,3,0,0,0,0,0,0
6,1604,2009091305,HOU,NYJ,HOU,home,NYJ,NYJ,2009-09-13,201,201.0,2001.0,Half1,NaN,0,kickoff,0,0,10,0,10,-10,0,0,0,0,0,0
7,3259,2009091306,IND,JAC,JAC,away,IND,JAC,2009-09-13,369,369.0,369.0,Half2,3.0,0,run,7,14,12,12,14,-2,0,0,0,0,0,0
8,3956,2009091302,CAR,PHI,PHI,away,CAR,CAR,2009-09-13,309,309.0,309.0,Half2,3.0,0,run,5,10,36,36,10,26,0,0,0,0,0,0
9,2547,2009091309,ARI,SF,SF,away,ARI,SF,2009-09-13,819,1719.0,1719.0,Half2,4.0,0,punt,0,6,13,13,6,7,0,0,0,0,0,0


In [47]:
df_nfl

DataFrame[play_id: int, game_id: int, home_team: string, away_team: string, posteam: string, posteam_type: string, defteam: string, side_of_field: string, game_date: timestamp, quarter_seconds_remaining: int, half_seconds_remaining: double, game_seconds_remaining: double, game_half: string, down: int, goal_to_go: int, play_type: string, yards_gained: int, total_home_score: int, total_away_score: int, posteam_score: int, defteam_score: int, score_differential: int, fourth_down_converted: int, fourth_down_failed: int, touchdown: int, pass_touchdown: int, rush_touchdown: int, return_touchdown: int]

In [48]:
df_nfl.toPandas()['play_id'].nunique()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


5311

In [49]:
df_nfl.toPandas().shape

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


(446982, 28)

In [50]:
#let's also see the number of partitions
rdd_nfl = df_nfl.rdd
rdd_nfl.getNumPartitions()

4

In [51]:
#convert the dataset to a RDD
rdd_nfl = rdd_nfl.repartition(4)
rdd_nfl.take(5)

[Row(play_id=3862, game_id=2009091309, home_team='ARI', away_team='SF', posteam='SF', posteam_type='away', defteam='ARI', side_of_field='ARI', game_date=datetime.datetime(2009, 9, 13, 0, 0), quarter_seconds_remaining=449, half_seconds_remaining=449.0, game_seconds_remaining=449.0, game_half='Half2', down=3, goal_to_go=1, play_type='pass', yards_gained=3, total_home_score=16, total_away_score=19, posteam_score=13, defteam_score=16, score_differential=-3, fourth_down_converted=0, fourth_down_failed=0, touchdown=1, pass_touchdown=1, rush_touchdown=0, return_touchdown=0),
 Row(play_id=1626, game_id=2009091311, home_team='SEA', away_team='STL', posteam='SEA', posteam_type='home', defteam='STL', side_of_field='STL', game_date=datetime.datetime(2009, 9, 13, 0, 0), quarter_seconds_remaining=350, half_seconds_remaining=350.0, game_seconds_remaining=2150.0, game_half='Half1', down=2, goal_to_go=1, play_type='pass', yards_gained=1, total_home_score=6, total_away_score=0, posteam_score=0, defteam_

Now we have the RDD converted from Dataframe so we can do RDD operations with `rdd_nfl`. Now let's replicate the two calculations (1) number of plays in each game (2) average yarns gained in each game.

In [52]:
#mapreduce with spark RDD for sum of plays
#[Your Code]
sum_plays = rdd_nfl.map(lambda row: 1)
sum_plays.reduce(lambda x,y : x+y)

446982

In [70]:
#MAPREDUCE
avg_plays = rdd_nfl.filter(lambda row: row['yards_gained'] != None)

avg_plays = avg_plays.map(lambda row: row['yards_gained'])
total = len(avg_plays.collect())
print(total)
avg_plays.reduce(lambda x,y: x+y)/total


446769


3.9018217468087535

Next, let's do it with spark dataframe and SQL. The dataframe is `df_nfl`.

In [72]:
df_nfl.createOrReplaceTempView("df_nfl")
df_nfl.show()

+-------+----------+---------+---------+-------+------------+-------+-------------+-------------------+-------------------------+----------------------+----------------------+---------+----+----------+---------+------------+----------------+----------------+-------------+-------------+------------------+---------------------+------------------+---------+--------------+--------------+----------------+
|play_id|   game_id|home_team|away_team|posteam|posteam_type|defteam|side_of_field|          game_date|quarter_seconds_remaining|half_seconds_remaining|game_seconds_remaining|game_half|down|goal_to_go|play_type|yards_gained|total_home_score|total_away_score|posteam_score|defteam_score|score_differential|fourth_down_converted|fourth_down_failed|touchdown|pass_touchdown|rush_touchdown|return_touchdown|
+-------+----------+---------+---------+-------+------------+-------+-------------+-------------------+-------------------------+----------------------+----------------------+---------+----+--

In [74]:
spark.sql('select count(game_id) from df_nfl').show()

+--------------+
|count(game_id)|
+--------------+
|        446982|
+--------------+



In [82]:
spark.sql('select avg(yards_gained) from df_nfl where yards_gained is NOT NULL').show()


+------------------+
| avg(yards_gained)|
+------------------+
|3.9018217468087535|
+------------------+



# DATA ANALYTICS WITH DATAFRAME API AND SPARK SQL
With the NFL Dataframe `df_nfl`, use either dataframe operations/API or spark SQL to answer the following questions.
First of all, let's build a data viewer to look at the data so we can understand the values better.

In [170]:
game_info_all = ['play_id', 'game_id', 'home_team', 'away_team', 'game_date', 
                 'posteam', 'posteam_type', 'defteam',
                 'total_home_score', 'total_away_score',
                 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown']

df_nfl.select(game_info_all).where('game_id = 2018111110').toPandas().head(200)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,play_id,game_id,home_team,away_team,game_date,posteam,posteam_type,defteam,total_home_score,total_away_score,touchdown,pass_touchdown,rush_touchdown,return_touchdown
0,1625,2018111110,LA,SEA,2018-11-11,LA,home,SEA,16,14,1.0,0.0,1.0,0.0
1,3478,2018111110,LA,SEA,2018-11-11,LA,home,SEA,35,24,1.0,0.0,1.0,0.0
2,3238,2018111110,LA,SEA,2018-11-11,LA,home,SEA,26,24,0.0,0.0,0.0,0.0
3,3449,2018111110,LA,SEA,2018-11-11,SEA,away,LA,29,24,0.0,0.0,0.0,0.0
4,3689,2018111110,LA,SEA,2018-11-11,SEA,away,LA,36,24,0.0,0.0,0.0,0.0
5,4058,2018111110,LA,SEA,2018-11-11,LA,home,SEA,36,31,0.0,0.0,0.0,0.0
6,1534,2018111110,LA,SEA,2018-11-11,LA,home,SEA,10,14,0.0,0.0,0.0,0.0
7,1183,2018111110,LA,SEA,2018-11-11,LA,home,SEA,10,14,0.0,0.0,0.0,0.0
8,2648,2018111110,LA,SEA,2018-11-11,LA,home,SEA,20,21,0.0,0.0,0.0,0.0
9,703,2018111110,LA,SEA,2018-11-11,SEA,away,LA,7,13,1.0,0.0,1.0,0.0



Lets find GAMES with highest number of plays from 2009 to 2018? And which game has the highest final score difference?

In [83]:
from pyspark.sql import functions as fn
from pyspark.sql import Window

#you need to show the game info with the highest plays, so let's obtain game level information
game_info = ['game_id', 'home_team', 'away_team', 'game_date', 'total_home_score', 'total_away_score']
#because we need the final scores for each game as game level info, we can do that by filtering the maxiumn play id to get game level info
window = Window.partitionBy('game_id')
nfl_game_info = df_nfl.withColumn("max_play_id", fn.max("play_id").over(window)).filter("max_play_id = play_id").drop("max_play_id").select(game_info)
nfl_game_info.show()

+----------+---------+---------+-------------------+----------------+----------------+
|   game_id|home_team|away_team|          game_date|total_home_score|total_away_score|
+----------+---------+---------+-------------------+----------------+----------------+
|2009091300|      ATL|      MIA|2009-09-13 00:00:00|              19|               7|
|2009091400|       NE|      BUF|2009-09-14 00:00:00|              25|              23|
|2009091401|      OAK|       SD|2009-09-14 00:00:00|              20|              24|
|2009092001|      DET|      MIN|2009-09-20 00:00:00|              13|              27|
|2009092004|       KC|      OAK|2009-09-20 00:00:00|              10|              13|
|2009092010|       SF|      SEA|2009-09-20 00:00:00|              23|              10|
|2009092011|      CHI|      PIT|2009-09-20 00:00:00|              17|              14|
|2009092012|      DEN|      CLE|2009-09-20 00:00:00|              27|               6|
|2009092013|       SD|      BAL|2009-09-20 

In [84]:
#get number of plays in each game
nfl_num_play = df_nfl.groupBy('game_id').agg(fn.count('play_id').alias('num_plays'))
nfl_num_play.show()

#join the two dataframes
nfl_game_info = nfl_game_info.join(nfl_num_play, 'game_id')

+----------+---------+
|   game_id|num_plays|
+----------+---------+
|2009092011|      164|
|2011103008|      181|
|2010110703|      157|
|2010112111|      161|
|2012111100|      178|
|2013110700|      173|
|2013101302|      186|
|2014092105|      177|
|2015110810|      202|
|2016102301|      165|
|2017102208|      163|
|2017112609|      188|
|2018092303|      174|
|2018111110|      184|
|2018120205|      171|
|2015121312|      180|
|2009121302|      166|
|2009122012|      163|
|2010100304|      165|
|2011010203|      170|
+----------+---------+
only showing top 20 rows



In [102]:
#[Your Code] to get the game with highest number of plays
nfl_game_info.createOrReplaceTempView("nfl_game_info")
spark.sql('select game_id from nfl_game_info where num_plays = (select max(num_plays) as max_num_plays from nfl_game_info)').show()
#game_id 2011120406 has highest number of plays

+----------+
|   game_id|
+----------+
|2011120406|
+----------+



In [103]:
#now it is the score difference
nfl_game_info = nfl_game_info.withColumn('score_diff', fn.abs(nfl_game_info['total_home_score'] - nfl_game_info['total_away_score']))

In [104]:
nfl_game_info.show()

+----------+---------+---------+-------------------+----------------+----------------+---------+----------+
|   game_id|home_team|away_team|          game_date|total_home_score|total_away_score|num_plays|score_diff|
+----------+---------+---------+-------------------+----------------+----------------+---------+----------+
|2009091300|      ATL|      MIA|2009-09-13 00:00:00|              19|               7|      161|        12|
|2009091400|       NE|      BUF|2009-09-14 00:00:00|              25|              23|      171|         2|
|2009091401|      OAK|       SD|2009-09-14 00:00:00|              20|              24|      184|         4|
|2009092001|      DET|      MIN|2009-09-20 00:00:00|              13|              27|      167|        14|
|2009092004|       KC|      OAK|2009-09-20 00:00:00|              10|              13|      183|         3|
|2009092010|       SF|      SEA|2009-09-20 00:00:00|              23|              10|      179|        13|
|2009092011|      CHI|      

In [143]:
#[Your Code] to get the game with highest score difference
nfl_game_info.select('game_id','score_diff').orderBy(fn.desc('score_diff')).show(1)

+----------+----------+
|   game_id|score_diff|
+----------+----------+
|2009101810|        59|
+----------+----------+
only showing top 1 row



2. On average how many plays are needed for a successful touchdown? And how many plays are needed for home team and away team, respectively?

In [145]:
nfl_game_play = df_nfl.groupBy('game_id').agg(fn.count('play_id').alias('total_plays'), fn.sum('touchdown').alias('total_touchdowns'))

In [160]:
nfl_game_play.select((fn.sum('total_plays')/fn.sum('total_touchdowns')).alias('average')).show()

+------------------+
|           average|
+------------------+
|35.049164902375914|
+------------------+



In [157]:
nfl_team_play = df_nfl.groupBy('game_id', 'posteam_type').agg(fn.count('play_id').alias('total_plays'), fn.sum('touchdown').alias('total_touchdowns'))

In [163]:
nfl_team_play.groupBy('posteam_type').agg((fn.sum('total_plays')/fn.sum('total_touchdowns')).alias('average')).show()

+------------+-----------------+
|posteam_type|          average|
+------------+-----------------+
|        null|             null|
|        away|35.75673887878287|
|        home| 32.5034297643901|
+------------+-----------------+



3. For touchdown, which type happened more likely on average, rush touchdown, pass touchdown or return touchdown? Are the probabilities different by home and away team?

In [176]:
total_touch_per_game = df_nfl.groupBy('game_id').agg(fn.sum('touchdown').alias('total_touchdown_game'), fn.sum('pass_touchdown').alias('total_pass_touchdown_game'),fn.sum('rush_touchdown').alias('total_rush_touchdown_game'),fn.sum('return_touchdown').alias('total_return_touchdown_game'))
total_touch_per_game.show(5)

+----------+--------------------+-------------------------+-------------------------+---------------------------+
|   game_id|total_touchdown_game|total_pass_touchdown_game|total_rush_touchdown_game|total_return_touchdown_game|
+----------+--------------------+-------------------------+-------------------------+---------------------------+
|2009092011|                   4|                        3|                        1|                          0|
|2011103008|                   4|                        4|                        0|                          0|
|2010110703|                   6|                        4|                        2|                          0|
|2010112111|                   3|                        2|                        1|                          0|
|2012111100|                   6|                        3|                        1|                          2|
+----------+--------------------+-------------------------+-------------------------+---

In [177]:
total_touch_per_game.select((fn.sum('total_pass_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_pass_td'),
                            (fn.sum('total_rush_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_rush_td'),
                            (fn.sum('total_return_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_return_td')).show()

+------------------+------------------+-------------------+
|   average_pass_td|   average_rush_td|  average_return_td|
+------------------+------------------+-------------------+
|0.6026033090253273|0.3131812122637811|0.05739825923312162|
+------------------+------------------+-------------------+



In [178]:
total_touch_per_game = df_nfl.groupBy('game_id','posteam_type').agg(fn.sum('touchdown').alias('total_touchdown_game'), fn.sum('pass_touchdown').alias('total_pass_touchdown_game'),fn.sum('rush_touchdown').alias('total_rush_touchdown_game'),fn.sum('return_touchdown').alias('total_return_touchdown_game'))
total_touch_per_game.show(5)

+----------+------------+--------------------+-------------------------+-------------------------+---------------------------+
|   game_id|posteam_type|total_touchdown_game|total_pass_touchdown_game|total_rush_touchdown_game|total_return_touchdown_game|
+----------+------------+--------------------+-------------------------+-------------------------+---------------------------+
|2010121213|        home|                   3|                        2|                        1|                          0|
|2009092003|        home|                   3|                        2|                        0|                          0|
|2010112106|        home|                   0|                        0|                        0|                          0|
|2010091912|        home|                   0|                        0|                        0|                          0|
|2011100909|        home|                   3|                        1|                        2|             

In [181]:
#avg by posteam_type
total_touch_per_game.groupBy('posteam_type').agg((fn.sum('total_pass_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_pass_td'),
                                                (fn.sum('total_rush_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_rush_td'),
                                                (fn.sum('total_return_touchdown_game')/fn.sum('total_touchdown_game')).alias('average_return_td')).show()

+------------+------------------+------------------+-------------------+
|posteam_type|   average_pass_td|   average_rush_td|  average_return_td|
+------------+------------------+------------------+-------------------+
|        null|              null|              null|               null|
|        away|0.6055895485364644|0.3016371754589052|0.06284107822060526|
|        home|0.5999105278854757|0.3235908141962422| 0.0524903071875932|
+------------+------------------+------------------+-------------------+



Teams with Highest win rate

In [182]:
#let's look at the available teams
df_nfl.select('home_team').distinct().show(50)

+---------+
|home_team|
+---------+
|      NYJ|
|      CAR|
|       TB|
|      OAK|
|      DET|
|      TEN|
|      BUF|
|      BAL|
|       NE|
|       GB|
|      JAC|
|      DEN|
|      ARI|
|       SF|
|       KC|
|      SEA|
|      CIN|
|      DAL|
|      CLE|
|      MIA|
|       SD|
|      STL|
|      MIN|
|      ATL|
|      PHI|
|      WAS|
|      NYG|
|      PIT|
|       NO|
|      IND|
|      HOU|
|      CHI|
|       LA|
|      JAX|
|      LAC|
+---------+



In [183]:
nfl_game_info = nfl_game_info.withColumn('win_team', fn.when(fn.col('total_home_score') > fn.col('total_away_score'), fn.col('home_team')).otherwise(fn.col('away_team')))
nfl_game_info = nfl_game_info.withColumn('game_year', fn.substring('game_date', 0, 4))
nfl_game_info.show()

+----------+---------+---------+-------------------+----------------+----------------+---------+----------+--------+---------+
|   game_id|home_team|away_team|          game_date|total_home_score|total_away_score|num_plays|score_diff|win_team|game_year|
+----------+---------+---------+-------------------+----------------+----------------+---------+----------+--------+---------+
|2009091300|      ATL|      MIA|2009-09-13 00:00:00|              19|               7|      161|        12|     ATL|     2009|
|2009091400|       NE|      BUF|2009-09-14 00:00:00|              25|              23|      171|         2|      NE|     2009|
|2009091401|      OAK|       SD|2009-09-14 00:00:00|              20|              24|      184|         4|      SD|     2009|
|2009092001|      DET|      MIN|2009-09-20 00:00:00|              13|              27|      167|        14|     MIN|     2009|
|2009092004|       KC|      OAK|2009-09-20 00:00:00|              10|              13|      183|         3|    

In [184]:
#create three sub dataframe, by team-year
win_count = nfl_game_info.groupBy(fn.col('win_team').alias('team'), 'game_year').agg(fn.count('win_team').alias('win_count'))
home_count = nfl_game_info.groupBy(fn.col('home_team').alias('team'), 'game_year').agg(fn.count('home_team').alias('home_count'))
away_count = nfl_game_info.groupBy(fn.col('away_team').alias('team'), 'game_year').agg(fn.count('away_team').alias('away_count'))

In [250]:
team_count = win_count.join(home_count, ['team','game_year'])
team_count = team_count.join(away_count,['team','game_year'])

In [251]:
team_count.show()

+----+---------+---------+----------+----------+
|team|game_year|win_count|home_count|away_count|
+----+---------+---------+----------+----------+
| ATL|     2012|       14|         9|         8|
| STL|     2012|        8|         9|         8|
| WAS|     2015|        8|         8|         7|
| WAS|     2014|        4|         8|         8|
| PHI|     2012|        5|         9|         8|
|  NE|     2013|       11|         8|         8|
|  GB|     2010|       11|         7|         9|
| PHI|     2010|       10|         7|         9|
| DET|     2012|        4|         8|         9|
| BAL|     2011|       12|         9|         7|
| DEN|     2012|       13|         9|         8|
| TEN|     2016|        8|         7|         9|
| DET|     2009|        2|         7|         8|
| MIN|     2009|       11|         7|         8|
| CLE|     2018|        6|         7|         7|
| OAK|     2010|        7|         9|         7|
| HOU|     2011|       11|         8|         8|
| SEA|     2015|    

In [252]:
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

In [253]:
team_count.createOrReplaceTempView("team_count")
spark.sql('select team, game_year, win_rate from team_count INNER JOIN (select game_year as game_year_new, max(win_rate) as win_rate_new from team_count GROUP BY game_year) as new on team_count.game_year = new.game_year_new and team_count.win_rate = new.win_rate_new ORDER BY game_year').show()

+----+---------+------------------+
|team|game_year|          win_rate|
+----+---------+------------------+
| IND|     2009|0.9333333333333333|
| ATL|     2010|            0.8125|
|  NE|     2010|            0.8125|
|  GB|     2011|            0.9375|
| ATL|     2012|0.8235294117647058|
| SEA|     2013|            0.8125|
| DEN|     2013|            0.8125|
|  NE|     2014|              0.75|
| DEN|     2014|              0.75|
|  GB|     2014|              0.75|
| SEA|     2014|              0.75|
| DAL|     2014|              0.75|
| CAR|     2015|0.9333333333333333|
|  NE|     2016|            0.8125|
| DAL|     2016|            0.8125|
|  NE|     2017|0.8235294117647058|
| PIT|     2017|0.8235294117647058|
| MIN|     2017|0.8235294117647058|
|  NO|     2018|0.8571428571428571|
+----+---------+------------------+

